# Motivation

The motivation for creating prediction and ranking tasks for EIE datasets was to extend the dataset ecosystem available for algorithmic fairness research in two directions: the education domain and the dataset from Eastern Europe (Ukraine). We obtained the data from publicly available Open Data resource https://zno.testportal.com.ua/opendata.


The tasks were created by Dr. Julia Stoyanovich and Andrew Bell from the Center for Responsible AI, New York University, and Tetiana Zakharchenko, Nazarii Drushchak, Oleksandra Konopatska, and Olha Liuba from Ukrainian Catholic University.


The creation of the dataset was funded by the Center for Responsible AI, New York University.

# Composition of Dataset

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

We will load and work with datasets from 2018 to 2021 since they have a comparable structure. We downloadedoriginal datasets from Open Data Resource and uploaded to HuggingFace for easiness of usage.

## Loading and merging

Load datasets from hugging face

In [73]:
# load datasets
datasets = []
years = range(2018, 2023)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.append(dataset)

EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_42787/3304867298.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='utf-8')


EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_42787/3304867298.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_42787/3304867298.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_42787/3304867298.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"zno/{year}/{file_name}", sep=";", encoding='utf-8')


EIE 2022 Loading...


Names of all features are the same across all years, but can have some difference with capital and lower cases. That's why we lower all of them. Also, we add a new column – a corresponding year.

In [74]:
#lowercase all columns and add year as attribute
for year, dataset in zip(years, datasets):
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year
    dataset['age'] = year - dataset['birth']

we drop from the list of datasets 2022 

In [75]:
nmt_df = datasets.pop()

Finally, we can concatenate all datasets.

In [76]:
# concat datasets
df_all = pd.concat(datasets)

For further research we add the column with age.

Save datasets to csv format.

In [77]:
df_all.to_csv('zno_all.csv',index=False)

Having created a dataset once, we can load it immediately in the future

In [48]:
df_all = pd.read_csv('zno_all.csv')

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_42787/2055058606.py:1: DtypeWarning: Columns (18,28,38,48,58,68,78,87,97,99,107,115,116,117,119,121,122,123,124,128,129,130,134,135,136,137,138,139,140,141,142,145,146,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv('zno_all.csv')


## Exploring dataset

In [78]:
df_all.head()

,outid,birth,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,classlangname,...,mathdpalevel,mathsttest,mathstlang,mathstteststatus,mathstball12,mathstball,mathstptname,mathstptregname,mathstptareaname,mathstpttername
0,a4e039d0-07ef-4a3d-820d-106dcaf01b96,2001,чоловіча,Закарпатська область,Берегівський район,с.Косонь,Випускник закладу загальної середньої освіти 2...,село,Фізико-математичний,угорська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c1fa1ed0-d2af-4964-9a88-3c138008304b,2001,жіноча,Київська область,Київська область,м.Бровари,Випускник закладу загальної середньої освіти 2...,місто,Іноземної філології,українська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6504a7f3-7dfd-4a64-a2d9-4d49f5021303,2000,жіноча,Харківська область,м.Харків,Червонозаводський район міста,Випускник закладу загальної середньої освіти 2...,місто,Іноземної філології,українська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5cbcd343-2e63-48b8-b36d-48797a76be31,2000,чоловіча,Рівненська область,Млинівський район,смт Млинів,Випускник закладу загальної середньої освіти 2...,місто,Фізико-математичний,українська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,063c6498-7345-492c-b567-174938bac671,1999,чоловіча,Рівненська область,Рівненська область,м.Острог,Учень (слухач) закладу професійної (професійно...,місто,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1458122 entries, 0 to 389322
Columns: 149 entries, outid to mathstpttername
dtypes: float64(28), int64(3), object(118)
memory usage: 1.6+ GB


In [80]:
dict(df_all.dtypes)

{'outid': dtype('O'),
 'birth': dtype('int64'),
 'sextypename': dtype('O'),
 'regname': dtype('O'),
 'areaname': dtype('O'),
 'tername': dtype('O'),
 'regtypename': dtype('O'),
 'tertypename': dtype('O'),
 'classprofilename': dtype('O'),
 'classlangname': dtype('O'),
 'eoname': dtype('O'),
 'eotypename': dtype('O'),
 'eoregname': dtype('O'),
 'eoareaname': dtype('O'),
 'eotername': dtype('O'),
 'eoparent': dtype('O'),
 'ukrtest': dtype('O'),
 'ukrteststatus': dtype('O'),
 'ukrball100': dtype('O'),
 'ukrball12': dtype('float64'),
 'ukrball': dtype('float64'),
 'ukrptname': dtype('O'),
 'ukrptregname': dtype('O'),
 'ukrptareaname': dtype('O'),
 'ukrpttername': dtype('O'),
 'histtest': dtype('O'),
 'histlang': dtype('O'),
 'histteststatus': dtype('O'),
 'histball100': dtype('O'),
 'histball12': dtype('float64'),
 'histball': dtype('float64'),
 'histptname': dtype('O'),
 'histptregname': dtype('O'),
 'histptareaname': dtype('O'),
 'histpttername': dtype('O'),
 'mathtest': dtype('O'),
 'mat

In [81]:
df_all.dropna(how = 'all', inplace = True)

In [82]:
df_all

,outid,birth,sextypename,regname,areaname,tername,regtypename,tertypename,classprofilename,classlangname,...,mathdpalevel,mathsttest,mathstlang,mathstteststatus,mathstball12,mathstball,mathstptname,mathstptregname,mathstptareaname,mathstpttername
0,a4e039d0-07ef-4a3d-820d-106dcaf01b96,2001,чоловіча,Закарпатська область,Берегівський район,с.Косонь,Випускник закладу загальної середньої освіти 2...,село,Фізико-математичний,угорська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c1fa1ed0-d2af-4964-9a88-3c138008304b,2001,жіноча,Київська область,Київська область,м.Бровари,Випускник закладу загальної середньої освіти 2...,місто,Іноземної філології,українська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6504a7f3-7dfd-4a64-a2d9-4d49f5021303,2000,жіноча,Харківська область,м.Харків,Червонозаводський район міста,Випускник закладу загальної середньої освіти 2...,місто,Іноземної філології,українська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5cbcd343-2e63-48b8-b36d-48797a76be31,2000,чоловіча,Рівненська область,Млинівський район,смт Млинів,Випускник закладу загальної середньої освіти 2...,місто,Фізико-математичний,українська,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,063c6498-7345-492c-b567-174938bac671,1999,чоловіча,Рівненська область,Рівненська область,м.Острог,Учень (слухач) закладу професійної (професійно...,місто,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389318,e62710ee-74fa-4085-b066-f38fc37ceae7,2003,жіноча,Рівненська область,Демидівський район,смт Демидівка,Учень (слухач) закладу професійної (професійно...,селище міського типу,Кваліфікований робітник,українська,...,стандарт(академічний),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389319,b2e4de44-7ebd-4163-8952-f988a9de1f67,1989,жіноча,Харківська область,Кегичівський район,смт Кегичівка,Учень (слухач) закладу професійної (професійно...,селище міського типу,Кваліфікований робітник,українська,...,NaN,Математика (завдання рівня стандарту),українська,Не з’явився,NaN,NaN,Красноградський ліцей №4 Красноградської міськ...,Харківська область,Красноградський район,м.Красноград
389320,e007c190-1fe1-4658-9eb0-f7cf489fc015,2004,жіноча,Одеська область,Одеська область,м.Білгород-Дністровський,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,...,NaN,Математика (завдання рівня стандарту),українська,Не з’явився,NaN,NaN,Загальноосвітня школа І-III ступенів № 1 м. Бі...,Одеська область,м.Білгород-Дністровський,м.Білгород-Дністровський
389321,2668b9f3-ddf9-4062-acd7-fdf6e2ca8cc8,2004,чоловіча,Вінницька область,Гайсинський район,с.Чечелівка,Випускник загальноосвітнього навчального закла...,"селище, село",Української філології,українська,...,профільний,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
for col in df_all.columns:
    if '100' in col:
        df_all[col] = pd.to_numeric(df_all[col].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))

In [93]:
dict(df_all.dtypes)

{'outid': dtype('O'),
 'birth': dtype('int64'),
 'sextypename': dtype('O'),
 'regname': dtype('O'),
 'areaname': dtype('O'),
 'tername': dtype('O'),
 'regtypename': dtype('O'),
 'tertypename': dtype('O'),
 'classprofilename': dtype('O'),
 'classlangname': dtype('O'),
 'eoname': dtype('O'),
 'eotypename': dtype('O'),
 'eoregname': dtype('O'),
 'eoareaname': dtype('O'),
 'eotername': dtype('O'),
 'eoparent': dtype('O'),
 'ukrtest': dtype('O'),
 'ukrteststatus': dtype('O'),
 'ukrball100': dtype('float64'),
 'ukrball12': dtype('float64'),
 'ukrball': dtype('float64'),
 'ukrptname': dtype('O'),
 'ukrptregname': dtype('O'),
 'ukrptareaname': dtype('O'),
 'ukrpttername': dtype('O'),
 'histtest': dtype('O'),
 'histlang': dtype('O'),
 'histteststatus': dtype('O'),
 'histball100': dtype('float64'),
 'histball12': dtype('float64'),
 'histball': dtype('float64'),
 'histptname': dtype('O'),
 'histptregname': dtype('O'),
 'histptareaname': dtype('O'),
 'histpttername': dtype('O'),
 'mathtest': dtype

In [97]:
df_all[df_all.ukrball100 == 0.0].ukrteststatus.value_counts()

ukrteststatus
Не подолав поріг    142770
Name: count, dtype: int64

## Unifying 'tertypename' feature

Since in 2021 'tertypename' has 3 territory types: city (місто), town (селище міського типу) and village (селище, село), and before we had only 2: city (місто) and village (селище, село), we reconstruct the correct type for 2018-2020 period.

In [98]:
def encode_tertypename(x):
    if x['tername'].startswith('м.'):
        return 'місто'
    elif x['tername'].startswith('смт'):
        return 'селище міського типу'
    elif x['tername'].startswith('с.'):
        return 'селище, село'
    elif x['tername'].startswith('с-ще'):
         return 'селище, село'
    elif x['areaname'].startswith('м.'):
        return 'місто'
    return None
        
df_all['tertypename'] = df_all.apply(encode_tertypename,axis=1)

In [99]:
df_all.tertypename.value_counts()

tertypename
місто                   1070663
селище, село             270811
селище міського типу     116648
Name: count, dtype: int64

## Unifying 'regtypename' feature

In [100]:
regtypename_unique = df_all.regtypename.unique()
sorted(regtypename_unique)

['Випускник загальноосвітнього навчального закладу 2021 року',
 'Випускник закладу загальної середньої освіти 2018 року',
 'Випускник закладу загальної середньої освіти 2019 року',
 'Випускник минулих років',
 'Випускник, який здобуде в 2018 році повну загальну середню освіту в навчальному закладі іншої держави',
 'Випускник, який здобуде в 2019 році повну загальну середню освіту в закордонному закладі освіти',
 'Випускник, який здобуде в 2020 році повну загальну середню освіту в закордонному закладі освіти',
 'Випускник, який здобуде в 2021 році повну загальну середню освіту в навчальному закладі іншої держави',
 'Студент закладу вищої освіти',
 'Учень (слухач) закладу професійної (професійно-технічної) освіти',
 'випускник закладу загальної середньої освіти 2020 року']

In [101]:
regtypename_dict = {
    'Випускник української школи поточного року': ['Випускник загальноосвітнього навчального закладу 2016 року','Випускник загальноосвітнього навчального закладу 2017 року',
                         'Випускник загальноосвітнього навчального закладу 2021 року', 'Випускник закладу загальної середньої освіти 2018 року',
                         'Випускник закладу загальної середньої освіти 2019 року', 'випускник закладу загальної середньої освіти 2020 року'],
    'Випускник іноземної школи': ['Випускник, який здобуде в 2016 році повну загальну середню освіту в навчальному закладі іншої держави', 'Випускник, який здобуде в 2017 році повну загальну середню освіту в навчальному закладі іншої держави',
                                  'Випускник, який здобуде в 2018 році повну загальну середню освіту в навчальному закладі іншої держави', 'Випускник, який здобуде в 2019 році повну загальну середню освіту в закордонному закладі освіти',
                                  'Випускник, який здобуде в 2020 році повну загальну середню освіту в закордонному закладі освіти', 'Випускник, який здобуде в 2021 році повну загальну середню освіту в навчальному закладі іншої держави'],
    'Випускник коледжу': ['Учень (слухач) закладу професійної (професійно-технічної) освіти', 'Учень (слухач, студент) професійно-технічного, вищого навчального закладу'],
    'Студент закладу вищої освіти': ['Студент закладу вищої освіти'],
    'Випускник минулих років': ['Випускник минулих років']
}

In [102]:
def encode_regtypename(x):
    for type, possible_types in regtypename_dict.items():
        if x in possible_types:
            return type
    return None

df_all.regtypename = df_all.regtypename.map(encode_regtypename)

In [103]:
df_all.regtypename.unique()

array(['Випускник української школи поточного року', 'Випускник коледжу',
       'Студент закладу вищої освіти', 'Випускник минулих років',
       'Випускник іноземної школи'], dtype=object)

## Drop Math Standard test and UkrsubTest

It was new experiment in 2021, that failed since due to COVID, it was cancelled.

In [104]:
df_all = df_all.drop(columns=['mathsttest', 'mathstlang', 'mathstteststatus', 'mathstball12', 
                              'mathstball','mathstptname','mathstptregname', 'mathstptareaname', 'mathstpttername', 'mathdpalevel'])

In [105]:
df_all = df_all.drop(columns=['ukrsubtest'])

## Change UML in 2021

In 2021 Ukrtest was renamed to UMLtest

In [106]:
def prep_ukr_uml(x, name):
    if x['year']<2021:
        return x[name]
    return x[name.replace('ukr','uml')]

for name in [col for col in df_all.columns if col.startswith('ukr')]:
    df_all[name] = df_all.apply(prep_ukr_uml,axis=1,args=[name])

In [107]:
df_all = df_all.drop(columns=[col for col in df_all.columns if col.startswith('uml')])

## Drop Ball12

They are ranking scores for different scales.

In [108]:
df_all = df_all.drop(columns=[col for col in df_all.columns if '12' in col])

In [109]:
df_all = df_all.drop(columns = 'birth')

In [110]:
list(df_all.columns)

['outid',
 'sextypename',
 'regname',
 'areaname',
 'tername',
 'regtypename',
 'tertypename',
 'classprofilename',
 'classlangname',
 'eoname',
 'eotypename',
 'eoregname',
 'eoareaname',
 'eotername',
 'eoparent',
 'ukrtest',
 'ukrteststatus',
 'ukrball100',
 'ukrball',
 'ukrptname',
 'ukrptregname',
 'ukrptareaname',
 'ukrpttername',
 'histtest',
 'histlang',
 'histteststatus',
 'histball100',
 'histball',
 'histptname',
 'histptregname',
 'histptareaname',
 'histpttername',
 'mathtest',
 'mathlang',
 'mathteststatus',
 'mathball100',
 'mathball',
 'mathptname',
 'mathptregname',
 'mathptareaname',
 'mathpttername',
 'phystest',
 'physlang',
 'physteststatus',
 'physball100',
 'physball',
 'physptname',
 'physptregname',
 'physptareaname',
 'physpttername',
 'chemtest',
 'chemlang',
 'chemteststatus',
 'chemball100',
 'chemball',
 'chemptname',
 'chemptregname',
 'chemptareaname',
 'chempttername',
 'biotest',
 'biolang',
 'bioteststatus',
 'bioball100',
 'bioball',
 'bioptname',
 '

In [111]:
df_all.to_csv('zno_preprocessed.csv', index=False)